In [1]:
import pandas as pd
import numpy as np
from pyhrv import time_domain as td
from pyhrv import frequency_domain as fd

In [2]:
df = pd.read_csv('HRV_Extractor.csv')

In [3]:
df['RR (ms)'] = 60000/df['bpm']

In [4]:
df

,bpm,time,set,creationDate,measureDate,RR (ms)
0,77,"21:50:57,93",0,2021-04-04 21:52:07 +0100,"2021-04-04 21:50:57,93",779.220779
1,66,"21:50:58,83",0,2021-04-04 21:52:07 +0100,"2021-04-04 21:50:58,83",909.090909
2,61,"21:50:59,82",0,2021-04-04 21:52:07 +0100,"2021-04-04 21:50:59,82",983.606557
3,66,"21:51:00,73",0,2021-04-04 21:52:07 +0100,"2021-04-04 21:51:00,73",909.090909
4,74,"21:51:01,55",0,2021-04-04 21:52:07 +0100,"2021-04-04 21:51:01,55",810.810811
...,...,...,...,...,...,...
815,80,"10:02:15,77",3,2021-04-05 10:02:26 +0100,"2021-04-05 10:02:15,77",750.000000
816,76,"10:02:16,56",3,2021-04-05 10:02:26 +0100,"2021-04-05 10:02:16,56",789.473684
817,76,"10:02:17,35",3,2021-04-05 10:02:26 +0100,"2021-04-05 10:02:17,35",789.473684
818,79,"10:02:18,11",3,2021-04-05 10:02:26 +0100,"2021-04-05 10:02:18,11",759.493671


In [5]:
def mean(df):
    return np.mean(df)
def SD(df):
    return np.std(df)
def max_(df):
    return np.max(df)
def min_(df):
    return np.min(df)
def RMSSD(df):
    return np.sqrt(np.mean(np.square(np.diff(df))))
def NN50(df):
    return np.sum(np.abs(np.diff(df)) > 50)*1
def pNN50(df):
    return 100 * np.sum((np.abs(np.diff(df)) > 50)*1) / len(df)

In [6]:
td = df.groupby(['set'])['RR (ms)'].apply(mean).to_frame(name = 'Mean RR (ms)').reset_index()
td['count'] = df.groupby(['set'])['RR (ms)'].count()
td['SDNN'] = df.groupby(['set'])['RR (ms)'].apply(SD)
td['Max RR (ms)'] = df.groupby(['set'])['RR (ms)'].apply(max_)
td['Min RR (ms)'] = df.groupby(['set'])['RR (ms)'].apply(min_)
td['Mean HR (bpm)'] = df.groupby(['set'])['bpm'].apply(mean)
td['SDHR'] = df.groupby(['set'])['bpm'].apply(SD)
td['Max HR (bpm)'] = df.groupby(['set'])['bpm'].apply(max_)
td['Min HR (bpm)'] = df.groupby(['set'])['bpm'].apply(min_)
td['RMSSD (ms)'] = df.groupby(['set'])['RR (ms)'].apply(RMSSD)
td['NN50'] = df.groupby(['set'])['RR (ms)'].apply(NN50)
td['pNN50 (%)'] = df.groupby(['set'])['RR (ms)'].apply(pNN50)

In [7]:
td

,set,Mean RR (ms),count,SDNN,Max RR (ms),Min RR (ms),Mean HR (bpm),SDHR,Max HR (bpm),Min HR (bpm),RMSSD (ms),NN50,pNN50 (%)
0,0,819.043960,74,119.261714,1090.909091,625.000000,74.864865,11.145626,96,55,93.413735,42,56.756757
1,1,698.926132,415,60.507018,857.142857,550.458716,86.493976,7.534616,109,70,37.369907,60,14.457831
2,2,690.202372,171,53.905513,909.090909,555.555556,87.444444,6.629820,108,66,50.566740,31,18.128655
3,3,704.831099,160,49.948913,882.352941,600.000000,85.537500,5.822250,100,68,40.657472,28,17.500000


In [8]:
%%capture
freq = df.groupby(['set'])['RR (ms)'].apply(fd.welch_psd)

In [9]:
freq.to_csv("freq.csv", header=None, index=False)

In [10]:
freq = pd.read_csv('freq.csv', sep=', ', header=None)

<ipython-input-10-c603d9dd2208>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  freq = pd.read_csv('freq.csv', sep=', ', header=None)


In [11]:
VLF = freq[10].str.split("(", n=1, expand=True)
HF = freq[12].str.split(")", n=1, expand=True)
LF_HF = freq[21].str.split("=", n=1, expand=True)

In [12]:
fd = pd.DataFrame()
fd['VLF'] = VLF[1]
fd['LF'] = freq[11]
fd['HF'] = HF[0]
fd['LF/HF'] = LF_HF[1]

In [13]:
fd

,VLF,LF,HF,LF/HF
0,450.46692621238986,15241.274233,2668.9155998272263,5.710661751319545
1,706.8764778540617,1187.509244,616.7035872138676,1.9255753790486216
2,192.93747775858293,1950.548400,613.2728474295423,3.180555617198278
3,449.13323462237963,770.368039,815.8855592175328,0.9442109011410801


In [14]:
df2 = pd.concat([td, fd], axis=1)

In [15]:
df2

,set,Mean RR (ms),count,SDNN,Max RR (ms),Min RR (ms),Mean HR (bpm),SDHR,Max HR (bpm),Min HR (bpm),RMSSD (ms),NN50,pNN50 (%),VLF,LF,HF,LF/HF
0,0,819.043960,74,119.261714,1090.909091,625.000000,74.864865,11.145626,96,55,93.413735,42,56.756757,450.46692621238986,15241.274233,2668.9155998272263,5.710661751319545
1,1,698.926132,415,60.507018,857.142857,550.458716,86.493976,7.534616,109,70,37.369907,60,14.457831,706.8764778540617,1187.509244,616.7035872138676,1.9255753790486216
2,2,690.202372,171,53.905513,909.090909,555.555556,87.444444,6.629820,108,66,50.566740,31,18.128655,192.93747775858293,1950.548400,613.2728474295423,3.180555617198278
3,3,704.831099,160,49.948913,882.352941,600.000000,85.537500,5.822250,100,68,40.657472,28,17.500000,449.13323462237963,770.368039,815.8855592175328,0.9442109011410801


In [16]:
df2.to_csv("HRV_Analysis.csv", index=False)